# HCIPy `SegmentedMirror()` vs. Poppy `HexSegmentedDeformableMirror()`

Both have the same functionality, but I need to confirm that hte HCIPy SM has the same results.  

Note that the Poppy HexDM display is currently broken and not the entire mirror is displayed, it is scaled weirdly. The simulator and the resulting PSFs work fine though.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.units as u
from hcipy import *
import poppy

from hcipy.optics.segmented_mirror import get_atlast_aperture, SegmentedMirror

In [ ]:
# Parameters
NPIX = 1024
PUP_DIAMETER = 15.   # m
GAPSIZE = 0.01  # m
FLATTOFLAT = 2.3   # m

wvln = 638e-9
lamD = 20
samp = 4
norm = False

## Instantiate the SMs

### HCIPy SM: `hsm`

We need to generate a pupil grid for the aperture, and a focal grid and propagator for the focal plane images after the DM.

In [ ]:
# HCIPy grids and propagator
pupil_grid = make_pupil_grid(dims=NPIX, diameter=PUP_DIAMETER)
focal_grid = make_focal_grid(pupil_grid, samp, lamD, wavelength=wvln)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

We generate a segmented aperture for the segmented mirror, this sets the geometry, nuber of segments etc.

In [ ]:
# Generate an aperture
aper, seg_pos = get_atlast_aperture(normalized=norm)
aper = evaluate_supersampled(aper, pupil_grid, 2)

plt.figure(figsize=(15, 15))
imshow_field(aper)
plt.title('HCIPy aperture')

Now we make the segmented mirror. In order to be able to apply the SM to a plane, that plane needs to be a `Wavefront()`, which combines a `Field` - here the aperture - with a wavelength, here `wvln`.

In this example here, since the SM doesn't have any extra effects on the pupil since it's completely flat still, we don't actually have to apply the SM, although of course we could. The first time we apply an instance of the SM to a wavefront, this will take a moment because it needs to generate the arrays that it uses to aberrate the segments. These are then saved and not recomputed, unless we change the array size of the wavefront.

In [ ]:
# Instantiate the segmented mirror
hsm = SegmentedMirror(aper, seg_pos)

# Make a pupil plane wavefront from aperture
wf = Wavefront(aper, wavelength=wvln)

# Apply SM if you want to
#wf = hsm(wf)

imshow_field(wf.intensity)
plt.title('Wavefront intensity at HCIPy SM')

### Poppy SM: `psm`

In [ ]:
psm = poppy.dms.HexSegmentedDeformableMirror(name='Poppy SM',
                                             rings=3,
                                             flattoflat=FLATTOFLAT*u.m,
                                             gap=GAPSIZE*u.m,
                                             center=False)

In [ ]:
# Display the transmission and phase of the poppy sm
plt.figure(figsize=(16, 8))
psm.display(what='both')

The segment gaps are not very well resolved, but they are there.

## Create reference images

### HCIPy reference image

We need to apply the SM to the wavefront in the pupil plane and then propagate it to the image plane.

In [ ]:
# Apply SM to pupil plane wf
wf_sm = hsm(wf)

# Propagate from SM to image plane
im_ref_hc = prop(wf_sm)

In [ ]:
# Display intensity and phase in image plane
plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after HCIPy SM')

# Get normalization factor for HCIPy reference image
norm_hc = np.max(im_ref_hc.intensity)

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_ref_hc.intensity/norm_hc))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im_ref_hc.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

print('HCIPy PSF shape: {}'.format(im_ref_hc.intensity.shaped.shape))

### Poppy reference image

For the Poppy propagation, we need to make an optical system of which we then calculate the PSF.  

I will try to match the image resolution and size of the HCIPy image. I first adjust the `pixelscale` and `fov_arcsec` such that their ratio works and then I add a tweak factor `fac` to scale it to the HCIPy image. I also set `oversample` to something that matches the HCIPy sampling (it's close enough). I keep reusing these numbers and the tweak factor later on in the notebook.

In [ ]:
# Make an optical system with the Poppy SM and a detector
psm.flatten()
osys = poppy.OpticalSystem()
osys.add_pupil(psm)
fac = 6.55      # I'm tweaking pixelscale and fov_arcsec to match the HCIPy image
pxscle = 0.0031*fac
fovarc = 0.05*fac
osys.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=10)

In [ ]:
# Calculate the PSF
psf = osys.calc_psf(wvln)
plt.figure(figsize=(10, 10))
poppy.display_psf(psf, vmin=1e-9, vmax=0.1)

# Get the PSF as an array
im_ref_pop = psf[0].data
print('Poppy PSF shape: {}'.format(im_ref_pop.shape))

# Get normalization from Poppy reference image
norm_pop = np.max(im_ref_pop)

### Display the two reference images next to each other

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_ref_hc.intensity/norm_hc))
plt.title('HCIPy reference PSF')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(np.log10(im_ref_pop/norm_pop))
plt.title('Poppy reference PSF')
plt.colorbar()

I'm afraid that the two images are just not *quite* scaled the same way, they're probably off by a pixel or two, but I'll try to take a difference anyway.

In [ ]:
ref_dif = im_ref_pop - im_ref_hc.intensity.shaped

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(ref_dif)
plt.title('Full image')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(ref_dif[60:100,60:100])
plt.title('Zoomed in')
plt.colorbar()

The difference actually looks symmetric, so I think I managed to scale the images the same. And the residual is on the order of 5e-16, I believe that's close enough to zero. But I wouldn't rely too much on this.

## Piston aberrations on a pair of segments

Remember that:  
- Poppy takes the OPD in astropy quantities, HCIPy in plain numbers in meters - and in surface error as opposed to OPD - that makes a factor of two difference
- Poppy's phase display is in OPD in meters, HCIPy in radians of phase

In [ ]:
# Define function from rad of phase to m OPD
def aber_to_opd(aber_rad, wvln):
    aber_m = aber_rad * wvln / (2 * np.pi)
    return aber_m
    
aber_rad = 6.2

print('Aberration: {} rad'.format(aber_rad))
print('Aberration: {} m'.format(aber_to_opd(aber_rad, wvln)))

### Apply the aberration to two segments

Remember that the segment numbering is different in HCIPy and in Poppy, they're off by 60 degrees.

In [ ]:
# Flatten both SMs just to be sure
hsm.flatten()
psm.flatten()

# HCIPy
for i in [19, 28]:
    hsm.set_segment(i, aber_to_opd(aber_rad, wvln), 0, 0)
    
# Poppy
for i in [34, 25]:
    psm.set_actuator(i,  aber_to_opd(aber_rad, wvln)*u.m, 0, 0)   # 34 in poppy is 19 in HCIPy

In [ ]:
# Display the HCIPy pistoned segments
wf_pistoned = hsm(wf)

imshow_field(wf_pistoned.phase, mask=aper, cmap='RdBu')
plt.title('Phase for HCIPy SM')
plt.colorbar()

In [ ]:
# Display the Poppy pistoned segments
plt.figure(figsize=(10,10))
psm.display(what='opd')

### Propagate to image plane with pair of pistoned segments

In [ ]:
### HCIPy
# Apply SM to pupil plane wf
wf_fp_pistoned = hsm(wf)

# Propagate from SM to image plane
im_pistoned_hc = prop(wf_fp_pistoned)

### Poppy
# Make an optical system with the Poppy SM and a detector
osys = poppy.OpticalSystem()
osys.add_pupil(psm)
pxscle = 0.0031*fac      # I'm tweaking pixelscale and fov_arcsec to match the HCIPy image
fovarc = 0.05*fac
osys.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=10)

# Calculate the PSF
psf = osys.calc_psf(wvln)
plt.figure(figsize=(10, 10))

# Get the PSF as an array
im_pistoned_pop = psf[0].data

### Display intensity of both cases image plane
plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM for $\phi$ = ' + str(aber_rad) + ' rad')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_pistoned_hc.intensity/norm_hc))
plt.title('HCIPy pistoned pair')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(np.log10(im_pistoned_pop/norm_pop))
plt.title('Poppy pistoned pair')
plt.colorbar()

### Quantify that:

I will now do the same thing like above with the aberrated pair of segmens, but I will do that for many differen aberration values and plot the image degradation - I do that by simply summing the images (integrated energy).

In [ ]:
# Aberration range
aber_array = np.linspace(0, 2*np.pi, 50, True)
print('Aber in rad: \n{}'.format(aber_array))
print('Aber in m: \n{}'.format(aber_to_opd(aber_array, wvln)))

In [ ]:
### Apply pistons
hc_ims = []
pop_ims = []
for aber_rad in aber_array:

    # Flatten both SMs
    hsm.flatten()
    psm.flatten()

    # HCIPy
    for i in [19, 28]:
        hsm.set_segment(i, aber_to_opd(aber_rad, wvln)/2, 0, 0)   # hsm takes it in meters of SURFACE, not OPD

    # Poppy
    for i in [34, 25]:
        psm.set_actuator(i, aber_to_opd(aber_rad, wvln) * u.m, 0, 0)  # 34 in poppy is 19 in HCIPy

    ### Propagate to image plane
    ### HCIPy
    # Apply SM to pupil plane wf
    wf_fp_pistoned = hsm(wf)

    # Propagate from SM to image plane
    im_pistoned_hc = prop(wf_fp_pistoned)

    ### Poppy
    # Make an optical system with the Poppy SM and a detector
    osys = poppy.OpticalSystem()
    osys.add_pupil(psm)
    pxscle = 0.0031 * fac  # I'm tweaking pixelscale and fov_arcsec to match the HCIPy image
    fovarc = 0.05 * fac
    osys.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=10)

    # Calculate the PSF
    psf = osys.calc_psf(wvln)

    # Get the PSF as an array
    im_pistoned_pop = psf[0].data

    hc_ims.append(im_pistoned_hc.intensity.shaped/np.max(im_pistoned_hc.intensity))
    pop_ims.append(im_pistoned_pop/np.max(im_pistoned_pop))
    
hc_ims = np.array(hc_ims)
pop_ims = np.array(pop_ims)

In [ ]:
### Quantify with image sums
sum_hc = np.sum(hc_ims, axis=(1,2))
sum_pop = np.sum(pop_ims, axis=(1,2)) - 1.75    # The -1.75 is just there because I didn't bother about
                                                # image normalization too much, I'm being sloppy here.
plt.suptitle('Image degradation of SMs')
plt.plot(aber_array, sum_hc, label='HCIPy SM')
plt.plot(aber_array, sum_pop, label='Poppy SM')
plt.xlabel('phase aberration (rad)')
plt.ylabel('image sum')
plt.legend()
plt.show()

If we were to take out the `/2` from the aberration that goes into HCIPy, the two curves wouldn't overlap, but instead HCIPy would complete two cycles while Poppy does only one in the same range of aberrations.

## Piston abberations with same amplitude on many segments

Here I will do the same as above, applying uniform pistons on segments, but this time on more segments than just two.  

Remember:  
- the different segment numbering between the two
- the factor of two division to get the aberration in surface error for the HCIPy SM
- the astropy unit `u*m` for the Poppy SM OPD aberration

In [ ]:
aber_rad = 1.7
print('Aberration in rad: {}'.format(aber_rad))
print('Aberration in m: {}'.format(aber_to_opd(aber_rad, wvln)))

In [ ]:
### Put aberrations on both SMs
# Flatten both SMs
hsm.flatten()
psm.flatten()

# HCIPy
for i in [19, 28, 23, 16, 3, 35, 30, 8, 14, 18, 1, 32, 12]:
    hsm.set_segment(i, aber_to_opd(aber_rad, wvln) / 2, 0, 0)   # hsm takes it in meters of SURFACE, not OPD

# Poppy
for i in [34, 25, 21, 14, 1, 32, 27, 10, 16, 12, 3, 30, 18]:
    psm.set_actuator(i, aber_to_opd(aber_rad, wvln) * u.m, 0, 0)  # 34 in poppy is 19 in HCIPy

In [ ]:
### Display the two SMs
# HCIPy
wf_pistoned = hsm(wf)

imshow_field(wf_pistoned.phase, mask=aper, cmap='RdBu')
plt.title('Phase for HCIPy SM')
plt.colorbar()

In [ ]:
# Poppy
plt.figure(figsize=(10,10))
psm.display(what='opd')

Seems like our segmented mirrors have the measles.

In [ ]:
### Propagate to image plane
## HCIPy
# Apply SM to pupil plane wf
wf_fp_pistoned = hsm(wf)

# Propagate from SM to image plane
im_pistoned_hc = prop(wf_fp_pistoned)

## Poppy
# Make an optical system with the Poppy SM and a detector
osys = poppy.OpticalSystem()
osys.add_pupil(psm)
pxscle = 0.0031 * fac  # I'm tweaking pixelscale and fov_arcsec to match the HCIPy image
fovarc = 0.05 * fac
osys.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=10)

# Calculate the PSF
psf = osys.calc_psf(wvln)

# Get the PSF as an array
im_pistoned_pop = psf[0].data

In [ ]:
### Display intensity of both cases image plane
plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM for $\phi$ = ' + str(aber_rad) + ' rad of piston')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_pistoned_hc.intensity/norm_hc))
plt.title('HCIPy pistoned arangement')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(np.log10(im_pistoned_pop/norm_pop))
plt.title('Poppy pistoned arangement')
plt.colorbar()

## Tip aberration on random segments

This time, we feed the Poppy SM the tip aberration directly in radians - remember to add the astropy unit `u*rad`. Also, this means that we're feeding the aberration into HCIPy in radians straight - no conversion to meters with our little function. But we **do** divide by two to account for surface error vs. OPD.  

Also, note that the regime in which we will be able to see the aberration change will be in the regime of micro- to nanoradians.

In [ ]:
# Define aberration
aber_rad = 200e-9
print('Aberration in rad: {}'.format(aber_rad))

In [ ]:
### Put aberrations on both SMs
# Flatten both SMs
hsm.flatten()
psm.flatten()

# HCIPy
for i in [19, 28, 23, 16, 3, 35, 30, 8, 14, 18, 1, 32, 12]:
    hsm.set_segment(i, 0, aber_rad/2, 0)   # hsm takes it in meters of SURFACE, not OPD

# Poppy
for i in [34, 25, 21, 14, 1, 32, 27, 10, 16, 12, 3, 30, 18]:  # 34 in poppy is 19 in HCIPy
    psm.set_actuator(i, 0, aber_rad * u.rad, 0)

In [ ]:
### Display the two SMs
# HCIPy
wf_pistoned = hsm(wf)

imshow_field(wf_pistoned.phase, mask=aper, cmap='RdBu')
plt.title('Phase for HCIPy SM')
plt.colorbar()

In [ ]:
# Poppy
plt.figure(figsize=(10,10))
psm.display(what='opd')

In [ ]:
### Propagate to image plane
## HCIPy
# Apply SM to pupil plane wf
wf_fp_pistoned = hsm(wf)

# Propagate from SM to image plane
im_pistoned_hc = prop(wf_fp_pistoned)

## Poppy
# Make an optical system with the Poppy SM and a detector
osys = poppy.OpticalSystem()
osys.add_pupil(psm)
pxscle = 0.0031 * fac  # I'm tweaking pixelscale and fov_arcsec to match the HCIPy image
fovarc = 0.05 * fac
osys.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=10)

# Calculate the PSF
psf = osys.calc_psf(wvln)

# Get the PSF as an array
im_pistoned_pop = psf[0].data

In [ ]:
### Display intensity of both cases image plane
plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM for $\phi$ = ' + str(aber_rad) + ' rad of tip')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_pistoned_hc.intensity/norm_hc))
plt.title('HCIPy tipped arangement')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(np.log10(im_pistoned_pop/norm_pop))
plt.title('Poppy tipped arangement')
plt.colorbar()

### Quanify that:

In [ ]:
# Aberration range
aber_array = np.linspace(0, 3000e-9, 50, True)
print('Aber in rad: \n{}'.format(aber_array))

In [ ]:
### Apply tips
hc_ims = []
pop_ims = []
for aber_rad in aber_array:

    # Flatten both SMs
    hsm.flatten()
    psm.flatten()

    # HCIPy
    for i in [19, 28]:
        hsm.set_segment(i, 0, aber_rad/2, 0)   # hsm takes it in meters of SURFACE, not OPD

    # Poppy
    for i in [34, 25]:
        psm.set_actuator(i, 0, aber_rad*u.rad, 0)  # 34 in poppy is 19 in HCIPy

    ### Propagate to image plane
    ### HCIPy
    # Apply SM to pupil plane wf
    wf_fp_pistoned = hsm(wf)

    # Propagate from SM to image plane
    im_pistoned_hc = prop(wf_fp_pistoned)

    ### Poppy
    # Make an optical system with the Poppy SM and a detector
    osys = poppy.OpticalSystem()
    osys.add_pupil(psm)
    pxscle = 0.0031 * fac  # I'm tweaking pixelscale and fov_arcsec to match the HCIPy image
    fovarc = 0.05 * fac
    osys.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=10)

    # Calculate the PSF
    psf = osys.calc_psf(wvln)

    # Get the PSF as an array
    im_pistoned_pop = psf[0].data

    hc_ims.append(im_pistoned_hc.intensity.shaped/np.max(im_pistoned_hc.intensity))
    pop_ims.append(im_pistoned_pop/np.max(im_pistoned_pop))

hc_ims = np.array(hc_ims)
pop_ims = np.array(pop_ims)

In [ ]:
### Quantify with image sums
sum_hc = np.sum(hc_ims, axis=(1,2))
sum_pop = np.sum(pop_ims, axis=(1,2)) - 1.75   # -1.75 again for flaud normalization. I'm being sloppy here.

plt.suptitle('Image degradation of SMs')
plt.plot(aber_array, sum_hc, label='HCIPy SM')
plt.plot(aber_array, sum_pop, label='Poppy SM')
plt.xlabel('phase aberration (rad)')
plt.ylabel('image sum')
plt.legend()
plt.show()

Note how the graphs of the two will overlap if we remember to account for the extra factor fo two because HCIPy works in surface and Poppy in OPD. There's still a slight offset though, not sure why.

## A mix of piston, tip and tilt (PTT)

In [ ]:
aber_rad_tt = 500e-9
aber_rad_p = 1.8

In [ ]:
### Put aberrations on both SMs
# Flatten both SMs
hsm.flatten()
psm.flatten()

## PISTON
# HCIPy
for i in [19, 28, 23, 16]:
    hsm.set_segment(i, aber_to_opd(aber_rad_p, wvln) / 2, 0, 0)   # hsm takes it in meters of SURFACE, not OPD
    
for i in [3, 35, 30, 8]:
    hsm.set_segment(i, aber_to_opd(aber_rad_p, wvln) / 2 /2, 0, 0)   # hsm takes it in meters of SURFACE, not OPD
    
for i in [14, 18, 1, 32, 12]:
    hsm.set_segment(i, aber_to_opd(aber_rad_p, wvln) / 2 /3, 0, 0)   # hsm takes it in meters of SURFACE, not OPD

# Poppy
for i in [34, 25, 21, 14]:  # 34 in poppy is 19 in HCIPy
    psm.set_actuator(i, aber_to_opd(aber_rad_p, wvln) * u.m, 0, 0)
    
for i in [1, 32, 27, 10]:  # 34 in poppy is 19 in HCIPy
    psm.set_actuator(i, aber_to_opd(aber_rad_p, wvln) * u.m /2, 0, 0)
    
for i in [16, 12, 3, 30, 18]:  # 34 in poppy is 19 in HCIPy
    psm.set_actuator(i, aber_to_opd(aber_rad_p, wvln) * u.m /3, 0, 0)
    
## TIP and TILT
# HCIPy
for i in [2, 5, 11, 15, 22]:
    hsm.set_segment(i, 0, aber_rad_tt / 2 /2, aber_rad_tt / 2 /3)
    
for i in [4, 6, 36]:
    hsm.set_segment(i, 0, aber_rad_tt / 2, 0)
    
for i in [34, 31, 7]:
    hsm.set_segment(i, 0, 0, aber_rad_tt / 2 *1.3)
    
# Poppy
for i in [2, 5, 7, 15, 22]:
    psm.set_actuator(i, 0, aber_rad_tt/2, aber_rad_tt/3)
    
for i in [6, 4, 26]:
    psm.set_actuator(i, 0, aber_rad_tt, 0)
    
for i in [28, 31, 11]:
    psm.set_actuator(i, 0, 0, aber_rad_tt*1.3)

In [ ]:
### Display the two SMs
# HCIPy
wf_pistoned = hsm(wf)

imshow_field(wf_pistoned.phase, mask=aper, cmap='RdBu')
plt.title('Phase for HCIPy SM')
plt.colorbar()

In [ ]:
# Poppy
plt.figure(figsize=(10,10))
psm.display(what='opd')

In [ ]:
### Propagate to image plane
## HCIPy
# Apply SM to pupil plane wf
wf_fp_pistoned = hsm(wf)

# Propagate from SM to image plane
im_pistoned_hc = prop(wf_fp_pistoned)

## Poppy
# Make an optical system with the Poppy SM and a detector
osys = poppy.OpticalSystem()
osys.add_pupil(psm)
pxscle = 0.0031 * fac  # I'm tweaking pixelscale and fov_arcsec to match the HCIPy image
fovarc = 0.05 * fac
osys.add_detector(pixelscale=pxscle, fov_arcsec=fovarc, oversample=10)

# Calculate the PSF
psf = osys.calc_psf(wvln)

# Get the PSF as an array
im_pistoned_pop = psf[0].data

In [ ]:
### Display intensity of both cases image plane
plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM forrandom arangement')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im_pistoned_hc.intensity/norm_hc))
plt.title('HCIPy random arangement')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(np.log10(im_pistoned_pop/norm_pop))
plt.title('Poppy tipped arangement')
plt.colorbar()

This is pretty awesome.